In [ ]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

In [ ]:
classes = os.listdir("animals")

In [ ]:
x=[]
y=[]
for i in classes:
    current_folder = os.path.join("animals",i)
    for j in os.listdir(current_folder):
        image_path = os.path.join("animals",i,j)
        image = cv2.imread(image_path)
        image = cv2.resize(image,(32,32))
        image = image.flatten()
        image = np.array(image)/255.0
        x.append(image)
        y.append(i)

In [ ]:
x=np.array(x)
print(x.shape)
y=np.array(y)
print(y.shape)

In [ ]:
# y=np.array(y)
(trainX, testX, trainY, testY) = train_test_split(x, y, test_size=0.25, random_state=42)

In [ ]:
lb = LabelBinarizer()
lb.fit(trainY)
trainY = lb.transform(trainY)
testY = lb.transform(testY)
classes_names = lb.classes_
print(classes_names)
classes_number = len(lb.classes_)
print(classes_number)

In [ ]:
model = Sequential()
model.add(Dense(1024, input_shape=(3072,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

In [ ]:
opt = SGD(lr=0.01)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
H = model.fit(x=trainX, y=trainY, validation_data=(testX, testY), epochs=100, batch_size=32)
predictions = model.predict(x=testX, batch_size=32)

In [ ]:
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))

In [ ]:
N = np.arange(0, 100)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy (Simple NN)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig("lallo.png")
cv2.imshow("lallo",cv2.imread("lallo.png"))
cv2.waitKey(0)
cv2.destroyAllWindows()